In [ ]:
import pandas as pd
import numpy as np

In [ ]:
board_games = pd.read_csv('dump/app_boardgame.csv')

categories = pd.read_csv('dump/app_category.csv')
categories_fk = pd.read_csv('dump/app_boardgamecategory.csv')

mechanics = pd.read_csv('dump/app_mechanic.csv')
mechanics_fk = pd.read_csv('dump/app_boardgamemechanic.csv')

In [ ]:
all_tables = [
    board_games,
    categories,
    categories_fk,
    mechanics,
    mechanics_fk,
]

for singular_table in all_tables:
    singular_table.drop(columns=['updated_at', 'created_at'], inplace=True)

board_games

In [ ]:
board_games.drop(columns=['image_url', 'description', 'year_published'], inplace=True)

In [ ]:
for index in categories_fk.index:
    cat_connection = categories.loc[categories['id'] == categories_fk.loc[index, 'category_id']]
    categories_fk.loc[index, 'category_id'] = cat_connection['name'].iloc[0]

categories_fk

In [ ]:
for index in mechanics_fk.index:
    mech_connection = mechanics.loc[mechanics['id'] == mechanics_fk.loc[index, 'mechanic_id']]
    mechanics_fk.loc[index, 'mechanic_id'] = mech_connection['name'].iloc[0]

mechanics_fk

In [ ]:
all_categories = [cat for cat in categories['name']]
all_categories

all_mechanics = [mech for mech in mechanics['name']]
all_mechanics

In [ ]:
for category in all_categories:
    board_games[category] = 0

for mechanic in all_mechanics:
    board_games[mechanic] = 0

column_max_vals = [
    {'column': 'min_players', 'max_value': board_games['min_players'].max()},
    {'column': 'max_players', 'max_value': board_games['max_players'].max()},
    {'column': 'age', 'max_value': board_games['age'].max()},
    {'column': 'min_playtime', 'max_value': board_games['min_playtime'].max()},
    {'column': 'max_playtime', 'max_value': board_games['max_playtime'].max()},
    {'column': 'rating', 'max_value': board_games['rating'].max()}
]

for index in board_games.index:
    for element in column_max_vals:
        board_games.loc[index, element['column']] = board_games.loc[index, element['column']] / element['max_value']

board_games

In [ ]:
for _, row in categories_fk.iterrows():
    board_game_id = row['board_game_id']
    category_name = row['category_id']

    board_games.loc[board_games['id'] == board_game_id, category_name] = 1

for _, row in mechanics_fk.iterrows():
    board_game_id = row['board_game_id']
    mechanic_name = row['mechanic_id']

    board_games.loc[board_games['id'] == board_game_id, mechanic_name] = 1

In [ ]:
board_games.drop(columns=['id', 'name'], inplace=True)

In [ ]:
from sklearn.cluster import KMeans
import seaborn as sns
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
recommendation_model = KMeans(n_clusters=5, verbose=1, random_state=1)

clusters = pd.DataFrame()
clusters['clusters'] = recommendation_model.fit_predict(board_games)

In [ ]:
pca_num_components = 2

reduced_data = PCA(n_components=pca_num_components).fit_transform(board_games)
results = pd.DataFrame(reduced_data, columns=['pca1','pca2'])

sns.scatterplot(x="pca1", y="pca2", hue=clusters['clusters'], data=results)
plt.title('K-means Clustering with 2 dimensions')
plt.show()

In [ ]:
selected_game = board_games.loc[56454]
selected_game = pd.DataFrame(selected_game).T
recommendation_model.predict(selected_game)

In [ ]:
import pickle

filename = 'recommendation_model.sav'
pickle.dump(recommendation_model, open(filename, 'wb'))